In [16]:
import requests as rq
import bs4 as bs
import selenium as sel
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import tldextract as tld
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time
import threading

In [2]:
sns.set_style('darkgrid')

In [17]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [4]:
driver.get("https://www.prageru.com/video/george-washington-a-general-without-an-army?playlist=american-presidents")

In [5]:
driver.maximize_window()

## Building the function for the individual videos' links


### Messing Around


In [142]:
buttons = driver.find_elements(By.TAG_NAME, "svg")
for button in buttons:
    if button.get_attribute('innerHTML') == "Facts & Sources":
        print(button)

In [143]:
buttons = driver.find_elements('xpath',
                               "//button[contains(@class, 'box-border inline-flex flex-col items-center justify-center gap-1 rounded-full px-0 py-[.625rem] text-xs font-normal focus:outline-none focus-visible:ring-4 disabled:opacity-60 sm:text-sm xl:flex-row xl:gap-2')]")

In [144]:
for button in buttons:
    if "Facts &amp; Sources" in button.get_attribute('innerHTML'):
        print(button.get_attribute("innerHTML"))
        button.click()

<svg class="inline-block" fill="currentColor" fill-rule="evenodd" focusable="false" role="img" height="20" width="20"><use xlink:href="#circle-check"></use></svg>Facts &amp; Sources


In [145]:
links = driver.find_elements('xpath',
                             "//a[contains(@class, 'box-border cursor-pointer items-center justify-center gap-2 rounded-full font-bold focus:outline-none focus-visible:ring-4 py-[.625rem] text-primary hover:opacity-80 focus-visible:ring-neutral-1/25 inline-flex px-0 text-sm')]")

In [146]:
sources = pd.DataFrame({
    "response_code": [],
    "sub_domain": [],
    "domain": [],
    "tld": []
})
for link in links:
    link = link.get_attribute("href")
    subdomain, domain, top_level = tuple(tld.extract(link))
    row = [rq.get(link).status_code, subdomain, domain, top_level]
    sources.loc[len(sources.index)] = row

## Building the function to collect the individual videos


In [226]:
videos = find_links_in_playlist(
    "https://www.prageru.com/video/george-washington-a-general-without-an-army?playlist=american-presidents")

In [227]:
fixed = []
for video in videos:
    fixed.append(video.get_attribute('href'))

In [228]:
sources = pd.DataFrame({
    "response_code": [],
    "sub_domain": [],
    "domain": [],
    "tld": []
})

counter = 0
for video in fixed:  # type: ignore
    video = video
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[1])
    get_sources(sources, video)
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

### The actual Function


In [5]:
def get_individual_source(link, dataframe):
    link = link.get_attribute('href')
    subdomain, domain, top_level = tuple(tld.extract(link))
    try:
        row = [rq.get(link).status_code, subdomain, domain, top_level]
    except:
        row = ['404', subdomain, domain, top_level]
    dataframe.loc[len(dataframe.index)] = row
        
    

In [11]:
def get_sources(driver, dataframe: pd.DataFrame, video_link: str):
    driver.get(video_link)
    buttons = driver.find_elements('xpath',
                                   "//button[contains(@class, 'box-border inline-flex flex-col items-center justify-center gap-1 rounded-full px-0 py-[.625rem] text-xs font-normal focus:outline-none focus-visible:ring-4 disabled:opacity-60 sm:text-sm xl:flex-row xl:gap-2')]")
    for button in buttons:
        if "Facts &amp; Sources" in button.get_attribute('innerHTML'):
            button.click()
    links = driver.find_elements('xpath',
                                 "//a[contains(@class, 'box-border cursor-pointer items-center justify-center gap-2 rounded-full font-bold focus:outline-none focus-visible:ring-4 py-[.625rem] text-primary hover:opacity-80 focus-visible:ring-neutral-1/25 inline-flex px-0 text-sm')]")
    for link in range(len(links)):
        link = links[link]
        link = link.get_attribute("href")
        subdomain, domain, top_level = tuple(tld.extract(link))
        try:
            row = [rq.get(link).status_code, subdomain, domain, top_level]
        except:
            row = ['404', subdomain, domain, top_level]
        dataframe.loc[len(dataframe.index)] = row

In [12]:
def find_links_in_playlist(driver, playlist_link: str):
    driver.get(playlist_link)
    time.sleep(2)
    videos = driver.find_elements('xpath',
                                  "//a[contains(@class, 'grid grid-cols-[142px_1fr] items-center gap-4 overflow-hidden sm:grid-cols-[168px_1fr]')]")
    return videos

In [13]:
def get_sources_of_playlist(driver, play_list_link: 'str', count: int = 0):
    sources = pd.DataFrame({
        "response_code": [],
        "sub_domain": [],
        "domain": [],
        "tld": []
    })

    videos = find_links_in_playlist(driver, play_list_link)

    fixed = []
    for video in videos:
        fixed.append(video.get_attribute('href'))

    number = 0
    for video in range(len(fixed)):
        video = fixed[video]
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])
        get_sources(driver, sources, video)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        number += 1
        if number == count:
            break
    return sources

In [14]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [15]:
sources = pd.DataFrame({
    "response_code": [],
    "sub_domain": [],
    "domain": [],
    "tld": []
})

print(get_sources(driver, sources,
            'https://www.prageru.com/video/theodore-roosevelt-the-action-hero-president?playlist=american-presidents'))

https://www.nps.gov/thrb/learn/historyculture/tr-rr-spanamwar.htm
https://www.amazon.com/United-States-Latin-America-Diplomacy-ebook/dp/B0BL5GS3WD/
https://www.newyorkalmanack.com/2022/02/william-jennings-bryan-theodore-roosevelt-jp-morgan/
https://millercenter.org/president/roosevelt/campaigns-and-elections
https://historynewsnetwork.org/blog/154562
https://www.theodorerooseveltcenter.org/Learn-About-TR/TR-Encyclopedia/Capitalism%20and%20Labor/Railroads
https://www.gilderlehrman.org/news/pure-food-and-drug-act-passed-day-1906
https://time.com/5259995/theodore-roosevelt-portrait-conservation-hunting/
https://www.theodorerooseveltcenter.org/Blog/Item/The%20US%20Forest%20Service%20is%20Established
https://www.nps.gov/thrb/learn/historyculture/trandthenpsystem.htm#:~:text=As%20President%20from%201901%20to,of%20Chickasaw%20National%20Recreation%20Area
https://www.theodorerooseveltcenter.org/Learn-About-TR/TR-Encyclopedia/Conservation/Antiquities%20Act%20of%201906
https://www.fws.gov/staff-

In [12]:
driver.get('https://www.prageru.com/video/god-vs-atheism-which-is-more-rational?playlist=religion-for-non-believers')

In [13]:
sources = get_sources_of_playlist(
    driver=driver,
    play_list_link='https://www.prageru.com/video/god-vs-atheism-which-is-more-rational?playlist=religion-for-non-believers',
)

100%|██████████| 22/22 [00:16<00:00,  1.34it/s]
0it [00:00, ?it/s]
100%|██████████| 31/31 [00:24<00:00,  1.24it/s]
0it [00:00, ?it/s]
100%|██████████| 32/32 [00:15<00:00,  2.10it/s]
0it [00:00, ?it/s]
100%|██████████| 16/16 [00:09<00:00,  1.72it/s]


In [14]:
driver.close()

In [15]:
sources['tld'].value_counts()

tld
com       411
org       130
edu        28
gov        24
            2
de          2
va          2
co.uk       2
gov.uk      1
net         1
org.uk      1
ac.uk       1
Name: count, dtype: int64

In [20]:
sources['domain'].value_counts()[:-10]

domain
amazon          106
prageru          69
biblegateway     30
peterkreeft      29
youtube          25
               ... 
prospect          1
www               1
city-journal      1
cuny              1
un                1
Name: count, Length: 129, dtype: int64